In [1]:
import numpy as np
import pandas as pd
import string
import re
import unicodedata
import nltk
import spacy
import statistics

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import collections

In [2]:
spacy_nlp = spacy.load("pt_core_news_sm")

In [3]:
df = pd.read_excel('data/BASE_ANALITICA.xlsx')

df = df[df['Resultado Sem Providência Publicação Gracco'] != 'Não Localizado']

df = df[['Resultado Sem Providência Publicação Gracco', 'Região', 'Memo do Andamento']]

df['Memo do Andamento'] = [str(s) for s in df['Memo do Andamento']]

# df['Resultado Sem Providência Publicação Gracco'] = [0 if res == 'NÃO' else 1 for res in df['Resultado Sem Providência Publicação Gracco']]

df['Memo do Andamento'] = [re.sub(r'(sem provid[eê]ncias|SEM PROVID[EÊ]NCIAS)', '', sent) for sent in df['Memo do Andamento']]

df.reset_index(inplace=True, drop=True)

df.shape

(38177, 3)

In [4]:
collections.Counter(df['Resultado Sem Providência Publicação Gracco'])

Counter({0: 17539, 1: 20638})

In [5]:
df[df['Memo do Andamento'].str.contains('SEM PROVIDÊNCIAS')].shape

(20518, 3)

In [6]:
df[df['Memo do Andamento'].str.contains('SEM PROVIDENCIAS')].shape

(3, 3)

In [7]:
def pre_spacy(frase):
    frase = str(frase)
    frase = re.sub(r'\s+', ' ', frase)
    return frase

In [8]:
def pos_spacy(frase):
    frase = str(frase)
    frase = frase.lower()
    frase = re.sub(r'¹|²|³|º|ª', '', frase)
    frase = frase.translate(str.maketrans('', '', string.punctuation))
    frase = re.sub(r'[^\w\s]|\d', '', frase)
    frase = re.sub(r'\s+', ' ', frase)
    return frase

In [9]:
def limpar_frase_spacy(frase):
    doc = spacy_nlp(frase)
    temp = [(token.orth_, token.pos_) for token in doc if token.pos_ not in ['PROPN']]
    
    concat = ''
    for par in temp:
        item = pos_spacy(par[0])
        if(len(item) > 2):
            concat += item + " "
    return concat

In [10]:
# df['Memo do Andamento'] = df['Memo do Andamento'].apply(pre_spacy)
limpar_frase_spacy(pre_spacy(df['Memo do Andamento'][5]))

'intimação para exequentes distribuidora cep deodoro cep dados junho junho cumprimento dias metropolitana curitiba processual distribuidora módulo cep deodoro cep determina quando ante defiro desde por oportunamente arquivemse publiquese nlls digitalmente por extinta arq '

In [11]:
#df['Memo do Andamento'][16]

'INTIMAÇÃO ANEXADA NA ABA DE DOCUMENTOS. 2.0000455-2'

In [53]:
import pickle
x_temp = pickle.load(open("clean_x.pkl",'rb'))

# x_temp = df['Memo do Andamento'].apply(pre_spacy)
# x_temp = x_temp.apply(limpar_frase_spacy)
# pickle.dump(x_temp, open("clean_x.pkl",'wb'))


In [11]:
def limpar_string(frase):
    frase = str(frase)
    frase = frase.lower()
    frase = re.sub(r'(sem provid[eê]ncias)', '', frase)
    frase = re.sub(r'<\w+>|\(\w+\)|-\w+|[^\w\s]|\d|¹|²|³|º|ª', '', frase)
    frase = re.sub(r'\s+', ' ', frase)
    frase = frase.translate(str.maketrans('', '', string.punctuation))
    return frase

In [54]:
x_temp = df['Memo do Andamento'].apply(limpar_string)

In [87]:
def adv_tokenizer(incoming_string):
    # removing some line endings
    incoming_string = incoming_string.replace('\r', '')
    incoming_string = incoming_string.replace('\n', '')
    # tokenizing string
    doc_spacy = spacy_nlp(incoming_string)
    # making the lemmatization of the tokenized string (ignoring puntuations and words from the stopwords list)
    return [token.lemma_ for token in doc_spacy if not ((token.is_punct) or (token.is_space))]

In [89]:
import pickle
x_temp = pickle.load(open("clean_x_adv.pkl",'rb'))

# x_temp = df['Memo do Andamento'].apply(adv_tokenizer)
# pickle.dump(x_temp, open("clean_x_adv.pkl",'wb'))


In [4]:
custom_stopwords = ['janeiro', 'fevereiro', 'marco', 'março', 'abril', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 'segunda', 'terca', 'terça', 
                    'quarta', 'quinta', 'sexta', 'sabado', 'sábado', 'domingo']

In [63]:
df['length'] = df['Memo do Andamento'].apply(str).apply(len)
df.groupby('Região').mean().round()['length']

Região
Centro-Oeste    2346.0
Nordeste        2004.0
Norte           2353.0
Sudeste         1170.0
Sul             1949.0
Name: length, dtype: float64

In [91]:
print(x_temp.apply(len).mean())

257.65460879587187


In [79]:
df.groupby('Resultado Sem Providência Publicação Gracco').var()

,length
Resultado Sem Providência Publicação Gracco,
0,1.110935e+07
1,3.884553e+06


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import RegexpTokenizer
from sklearn.pipeline import Pipeline
from nltk.stem import RSLPStemmer
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn import utils
import unicodedata

from sklearn.linear_model import SGDClassifier

In [6]:
x_temp = df['Memo do Andamento']

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_temp, df['Resultado Sem Providência Publicação Gracco'], test_size=0.3, random_state=42)


In [10]:
# vect = TfidfVectorizer(strip_accents='unicode', stop_words=custom_stopwords)
vect = TfidfVectorizer(strip_accents='unicode', ngram_range=(1, 3),  min_df=0.2, max_df=0.8, stop_words=custom_stopwords)
        

clust = KMeans(n_clusters=3)

# clf = XGBClassifier(n_jobs=-1, n_estimators=100)
clf = RandomForestClassifier(n_estimators=100)
# clf = LogisticRegression()
# clf = SVC(gamma='scale', probability=False)
# clf = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
# clf = SGDClassifier(epsilon=0.01)


text_pipe = Pipeline([('vect', vect), ('clust', clust), ('clf', clf)])
# text_pipe = Pipeline([('vect', vect), ('clf', clf)])


In [21]:
vect = TfidfVectorizer(strip_accents='unicode', ngram_range=(1, 3),  min_df=0.2, max_df=0.8, stop_words=custom_stopwords)
clust = KMeans(n_clusters=3)
clf = RandomForestClassifier(n_estimators=100)

cv = cross_val_score(text_pipe, x_train, y_train, cv=5, verbose=1, n_jobs=-1)
# cv

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 10.5min finished


array([0.79812909, 0.79831618, 0.795884  , 0.79509731, 0.79079341])

In [8]:
x_train.head()

34034    \r\n\r\nDATA DE DISPONIBILIZACAO DA PUBLICACAO...
33127    \r\n\r\n1º JUIZADO ESPECIAL CÍVEL E CRIMINAL\r...
24967    LIDA EM: 12/06/2018. \r\n\r\n» DADOS DO PROCES...
29047    \r\nTURIAÇU\r\nDATA DE DISPONIBILIZACAO DA PUB...
22129     - PRAZOS JÁ AGENDADOS.\r\n\r\nVARA ESPECIALIZ...
Name: Memo do Andamento, dtype: object

In [9]:
clf = RandomForestClassifier(n_estimators=10)
text_pipe = Pipeline([('vect', vect), ('clf', clf)])


cv = cross_val_score(text_pipe, x_train, y_train, cv=5, verbose=1, n_jobs=-1)

cv

NameError: name 'vect' is not defined

In [53]:
cv.mean()

0.8348240072707718

In [51]:
cv.mean()

0.8564905642410221

In [11]:
clf = RandomForestClassifier(n_estimators=10)
text_pipe = Pipeline([('vect', vect), ('clf', clf)])


cv = cross_val_score(text_pipe, x_train, y_train, cv=5, verbose=1, n_jobs=-1)

cv

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.1min finished


array([0.82918616, 0.83610851, 0.82993452, 0.84206587, 0.84281437])

In [12]:
cv.mean()

0.8360218851352487

In [22]:
text_pipe.fit(x_train, y_train)

/home/marcelo/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=0.2,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [23]:
pred = text_pipe.predict(x_test)

In [24]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.66      0.44      0.53      5281
           1       0.63      0.81      0.71      6173

   micro avg       0.64      0.64      0.64     11454
   macro avg       0.64      0.62      0.62     11454
weighted avg       0.64      0.64      0.62     11454



In [25]:
print(confusion_matrix(y_test, pred))

[[2326 2955]
 [1192 4981]]


In [450]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         NÃO       0.99      0.99      0.99      5281
         SIM       0.99      0.99      0.99      6173

   micro avg       0.99      0.99      0.99     11454
   macro avg       0.99      0.99      0.99     11454
weighted avg       0.99      0.99      0.99     11454



In [451]:
print(confusion_matrix(pred, y_test))

[[5204   31]
 [  77 6142]]
